# Google Scholar 
By using lookup fields:

<a href="http://bit.ly/gslookup">https://scholar.google.co.in/scholar_lookup?<font color="red">title=</font>Estimates+for+the+number+of+sums+and+products+and+for+exponential+sums+in+fields+of+prime+order&<font color="red">author=</font>Jean+Bourgain&author=AA+Glibichuk&<font color="red">author=</font>SERGEI+VLADIMIROVICH+Konyagin&<font color="red">year=</font>2006&<font color="red">doi=</font>10.1112/S0024610706022721&<font color="red">publisher=</font>Oxford+University+Press&<font color="red">journal=</font>Journal+of+the+London+Mathematical+Society&<font color="red">volume=</font>73&<font color="red">issue=</font>2&<font color="red">pages=</font>380-398&hl=en</a>

In [1]:
import Levenshtein # pip3 install python-levenshtein
import re
from bs4 import BeautifulSoup
import requests

url='https://scholar.google.com'

import requests
headers_Get = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:49.0) Gecko/20100101 Firefox/49.0',
            'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
            'Accept-Language': 'en-US,en;q=0.5',
            'Accept-Encoding': 'gzip, deflate',
            'DNT': '1',
            'Connection': 'keep-alive',
            'Upgrade-Insecure-Requests': '1'
        }
r=requests.Session()

#import googlescholar as gs
#%%writefile ../cienciometria/googlescholar.py

def firefox_get(url):

    rget=r.get(url,headers=headers_Get)
    
    html = rget.text
    if html.lower().find('gs_captcha_f')>-1:
        sys.exit('ERROR: Captcha anti-robots requested!\n Aborting execution.')

    return html

def get_google_scholar(record):
    '''
    Analyise the BeautifulSoup record for an article 
    in Google Scholar.
    Output is a Python dictionary with keys: 
    'title', 'authors','profiles','Jornal','Year',
    'abstract','cites','cites_link'
    '''
    import random
    import time
    gsr={}
    try:
        cites=record.find_all('a',{"href":re.compile( "/scholar\?cites=" )})[0]
        try:
            gsr['cites']=int( cites.text.split()[-1] )
            gsr['cites_link']=cites.attrs.get('href')
        except:
            gsr['cites']=0
    except:
        cites=None

    # Title
    try:
        #The .split('XXX')[-1]  does not afect the result when .text does no contains 'XXX'
        tc=record.find_all('h3',{"class":"gs_rt"})[0].text.split('[CITATION][C] ')[-1]
    except:
        tc=''

    gsr['title']=tc.strip().split('[HTML][HTML] ')[-1].split(
                                  '[PDF][PDF] '  )[-1]
    
    # Explore authors, google scholar profile - Journal, Year - Publisher
    gpa=None
    try:
        gpa=record.find_all('div',{"class":"gs_a"})[0]
        #Full ref with authors, google scholar profile - Journal, Year - Publisher
        ref=gpa.text
        gsr['ref']=ref.strip()
        refparts=ref.split('\xa0-')
    except IndexError:
        gsr['ref']=''
        refparts=[]

    try:
        gsr['authors']=refparts[0]
    except IndexError:    
        gsr['authors']=''
        
    try:
        journalparts=refparts[-1].strip().split(' - ')
        gsr['publisher']=journalparts[-1]
        gsr['Journal']=journalparts[0].split(',')[0]
        gsr['Year']=journalparts[0].split(',')[-1].strip()
    except IndexError:
        gsr['publisher']=''
        gsr['Journal']=''
        gsr['Year']

    #Abstract:
    try:
        gsr['abstract']=record.find_all('div',{'class':'gs_rs'})[0].text.replace('\xa0…','')
    except:
        gsr['abstract']=''
    # profiles
    if gpa:
        lpr=gpa.find_all("a",{ "href":re.compile("/citations\?user=*")   } )
        prf={}
        for pr in lpr:
            prf[ pr.text ]=pr.attrs.get('href').split('?')[-1].split('&')[0].split('user=')[-1]
        gsr['profiles']=prf
    
    time.sleep( random.randint(1,3)  ) # avoid robots
    return gsr

def request_google_scholar_url(url):
    return requests.get(url)

def google_scholar_page(html):
    '''
    Convert a Google Scholar page into a list
    of dictionaries with metadata info
    '''
    if html.lower().find('gs_captcha_f')>-1:
        input('check robots')
   
    soup = BeautifulSoup(html, "html.parser")
    rgs=soup.find_all('div', {'class':'gs_ri' })

    citations=[]
    for record in rgs:
        citations.append( get_google_scholar(record) )
        
    return citations

def google_scholar_query(title='', author='', coauthors=[], DOI='', year=0, publisher='',
                         journal='', volume='', issue='', pages=0,
                         DEBUG=False):
    '''
    Search Google scholar with `sholar_lookup` full fields.
    Only the first result is analized. The output includes 
    a quality measurements between the query and the results 
    Output is a Python dictionary with keys: 
    'title', 'authors','profiles','cites','cites_link',
    'quality_title','quality_author'
    '''
    gs={}
    # + → %2B in query formula:
    baseurl='https://scholar.google.com/'
    q='scholar_lookup?'
    
    nl=0
    if title:
        nl=nl+1
        q= q+'title={}'.format(title.replace(' ','+'))
    if author:
        if nl: q= q+'&'
        nl=nl+1
        q= q+'author={}'.format(author.replace(' ','+'))
    if DOI:
        if nl: q= q+'&'
        nl=nl+1    
        q= q+'doi={}'.format(DOI)
    if year:
        if nl: q= q+'&'
        nl=nl+1
        q= q+'year={}'.format(year)
    if publisher:
        if nl: q= q+'&'
        nl=nl+1
        q= q+'publisher={}'.format(publisher.replace(' ','+'))
    if journal:
        if nl: q= q+'&'
        nl=nl+1     
        q= q+'journal={}'.format(journal.replace(' ','+'))
    if volume:
        if nl: q= q+'&'
        nl=nl+1
        q= q+'volume={}'.format(volume)
    if issue:
        if nl: q= q+'&'
        nl=nl+1
        q= q+'issue={}'.format(issue)   
    if pages:
        if nl: q= q+'&'
        nl=nl+1
        q= q+'pages={}'.format(pages)
    if coauthors:
        for i in coauthors :
            if nl: q= q+'&'
            nl=nl+1
            q= q+'author={}'.format(i.replace(' ','+'))    
    url=baseurl+q
    if DEBUG:
        print(url)   
    
    #s = requests.Session()
    rtext=firefox_get(url)

    #soup = BeautifulSoup(r.text, "html.parser")
    soup = BeautifulSoup(rtext, "html.parser")


    # Main contents:
    rgs=soup.find_all('div', {'class':'gs_ri' })

    try:
        record=rgs[0]
    except IndexError:
        #exit if record not found and returns empty dictionary
        return gs
    
    gs.update(get_google_scholar(record))
    
    #Check if author is in authors list
    if gs and author:
        sau=0
        for a in gs['authors'].split(','):
            saun=Levenshtein.ratio(author.lower(),a.lower().strip())
            if saun>sau:
                sau=saun
                
        gs['quality_author']=round(sau,2)
    else:
        gs['quality_author']=-1 #-1 means not checked
        
    if gs and title:
        gs['quality_title']=round( Levenshtein.ratio(
                   title.lower(),gs['title'].lower() ),2 )
    else:
        gs['quality_title'] =-1 #-1 means not checked
        
    #EXTRA FIELDS:
    #PDF
    try:
        gs['PDF']=soup.find_all('div',
                        {"class":"gs_or_ggsm"})[0].find_all('a')[0].get("href")
    except:
        gs['PDF']=''


    if DEBUG:
        print('='*80)
        print(record)
        
        return gs,record
    else:
        return gs

def get_cites_refs(browser,url,maxcites=65,t=60):
    """
    WARNING: Works only with SELENIUM true
    """
    import random
    import time
    url='https://scholar.google.com'+url
    browser.get(url)
     
    endpage=int(maxcites/10)+1
    refs=''
    
    kk=google_scholar_page( browser.page_source )
    try:
        refs=refs+'\n'.join( list((pd.DataFrame(kk)['title']+'; '
                                  +pd.DataFrame(kk)['ref']).values) )+'\n' 
    except:
        refs=''
    
    
    for i in range(endpage):
        try:
            browser.find_element_by_class_name('gs_ico_nav_next').click()
            kk=google_scholar_page( browser.page_source )
            try:
                refs=refs+'\n'.join( list( (pd.DataFrame(kk)['title']+'; '
                                +pd.DataFrame(kk)['ref']).values ) )+'\n' 
            except:
                refs=''
        except:
            break
            
    time.sleep(random.uniform(0.9*t,1.1*t))
    return refs

##  Load database

In [2]:
import wosplus as wp

In [11]:
%%writefile drive.cfg
[FILES]
DOIS.xlsx               = 1bikNT7Gmp4G7dfeMuGsF-az7D8lskK0O
UDEA_WOS_SCI_SCP.xlsx   = 1o9otmklgh-0w18Avv2ZTKOXr3vZbjwvj
oaudea.xlsx             = 1CcwobiEFACIbffNzNdLxpdxQukr8cZ5x
oaudea.json             = 1BmqRoQDgfpOFjXBVEfI999uaN6XX6Fkd
datos1.csv              = 11CyLRZZwVbgw6YAC-igRJ3mrkIwk0aaiXnd-EOofYTI
RedalycMetadatosArticulos.csv=# Google API key

Overwriting drive.cfg


In [12]:
# USE Google Drive id here

In [13]:
oa=wp.wosplus('drive.cfg')

In [14]:
oaudea=oa.read_drive_csv('RedalycMetadatosArticulos.csv')

/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py:2903: DtypeWarning: Columns (6,12) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


In [15]:
oaudea=wp.fill_NaN(oaudea)

In [16]:
oaudea.shape

(629851, 13)

In [17]:
datos=oaudea.copy()

In [18]:
#JUST START AFTER FINISH REDALYC
nini=6000# initial doi
n=3000 # Total of DOIs
nend=nini+n
T=12 #hours of search
t=T/n*3600 # [s] query time
t=60
day=24*3600 #s
mintime=0.9*t*n # [s] minimal time search
wait=30#day-mintime # maximum wait

In [19]:
t=60

In [20]:
datos=datos[datos.DOI!=''].reset_index(drop=True)

In [21]:
datos.columns

Index(['REVISTA', 'INSTITUCION_REVISTA', 'PAIS_REVISTA', 'IDARTICULO',
       'TITULO', 'Autor(es)', 'DOI', 'ANIO', 'VOLUMEN', 'NUMERO', 'PAGINAS',
       'IDIOMA', 'RESUMEN'],
      dtype='object')

In [22]:
ii=1
(datos.loc[ii,'DOI'],
 datos.loc[ii,'TITULO'],
 datos.loc[ii,'Autor(es)'].split(', ')[0],
 datos.loc[ii,'Autor(es)'].split('\n')[1:4],
 datos.loc[ii,'ANIO'],
 datos.loc[ii,'REVISTA'],
)

('10.4067/S0717-554X2017000100013',
 'Ventajas del análisis sistémico aplicado a los espacios locales',
 'Manuela Ortega ,M. Concepción Segovia ',
 [],
 2017,
 'Cinta de Moebio')

In [23]:
datos.shape

(18815, 13)

In [24]:
datos[nini:n].index

RangeIndex(start=6000, stop=3000, step=1)

In [25]:
datos.loc[ii,'ANIO']

2017

In [26]:
import pandas as pd
import time
import random
doi='DOI'
title_simple='TITULO'
article_id='IDARTICULO'

d=google_scholar_query(title=datos.loc[ii,title_simple],
                    author=datos.loc[ii,'Autor(es)'].split(', ')[0],
                    coauthors=datos.loc[ii,'Autor(es)'].split(', ')[1:4],
                    DOI=datos.loc[ii,doi],
                    year=datos.loc[ii,'ANIO'],
                    journal=datos.loc[ii,'REVISTA'])
d

{'Journal': 'Cinta de moebio',
 'PDF': 'https://scielo.conicyt.cl/scielo.php?pid=S0717-554X2017000100013&script=sci_arttext',
 'Year': '2017',
 'abstract': 'This paper shows how system analysis applied to local spaces facilitate the generation of proposals for integral development in the field of sustainability. Regions are composed of physical and social components generating products that benefit or harm themselves. These resources can be exploited or remained idle for lack of actors to develop them. They react to internal and external actions, both human and natural. Among these actions are the policies for local development. For these policies to be efficient, they need comprehensive studies',
 'authors': 'M Ortega, MC Segovia',
 'cites': 1,
 'cites_link': '/scholar?cites=14403974261697570786&as_sdt=2005&sciodt=0,5&hl=en',
 'profiles': {'M Ortega': 'GkYFaLMAAAAJ'},
 'publisher': 'scielo.conicyt.cl',
 'quality_author': 0.42,
 'quality_title': 1.0,
 'ref': 'M Ortega, MC Segovia\xa0- 

### Main loop

In [ ]:
import pandas as pd
import time
import random
doi='DOI'
title_simple='TITULO'
article_id='IDARTICULO'
dfgs=pd.DataFrame()
ibrkn=0
maxibrn=400

for ii in datos[nini:nend].index:
    print(ii,datos.loc[ii,doi])
    #gsd=google_scholar_query(DOI=doi)
    gsd=google_scholar_query(title=datos.loc[ii,title_simple],
                    author=datos.loc[ii,'Autor(es)'].split(', ')[0],
                    coauthors=datos.loc[ii,'Autor(es)'].split(', ')[1:4],
                    DOI=datos.loc[ii,doi],
                    year=datos.loc[ii,'ANIO'],
                    journal=datos.loc[ii,'REVISTA'] )
            
    gsd['old_title']=datos.loc[ii,'TITULO']
    gsd['DOI']=datos.loc[ii,'DOI']
    gsd['ID_ARTICLE']=datos.loc[ii,article_id]
    dfgs=dfgs.append(gsd,ignore_index=True )
    dfgs.to_json('rdlycdoi_{}_{}.json'.format(nini,nend))
    time.sleep(random.uniform(0.9*t,1.1*t))

6000 10.7440/antipoda28.2017.03
6001 10.7440/antipoda28.2017.04
6002 10.7440/antipoda28.2017.05
6003 10.7440/antipoda28.2017.06
6004 10.7440/antipoda28.2017.07
6005 10.7440/antipoda28.2017.08
6006 10.7440/antipoda28.2017.09
6007 10.7440/antipoda28.2017.10
6008 10.7440/antipoda29.2017.01
6009 10.7440/antipoda29.2017.02
6010 10.7440/antipoda29.2017.03
6011 10.7440/antipoda29.2017.04
6012 10.7440/antipoda29.2017.05
6013 10.7440/antipoda29.2017.06
6014 10.7440/antipoda29.2017.07
6015 10.7440/antipoda29.2017.08
6016 10.7440/antipoda29.2017.09
6017 10.7440/antipoda29.2017.10
6018 10.7440/antipoda29.2017.11
6019 10.7440/antipoda30.2018.01
6020 10.7440/antipoda30.2018.02
6021 10.7440/antipoda30.2018.03
6022 10.7440/antipoda30.2018.04
6023 10.7440/antipoda30.2018.05
6024 10.7440/antipoda30.2018.06
6025 10.7440/antipoda30.2018.07
6026 10.7440/antipoda30.2018.08
6027 10.7440/antipoda31.2018.01
6028 10.7440/antipoda31.2018.02
6029 10.7440/antipoda31.2018.03
6030 10.7440/antipoda31.2018.04
6031 10.

6262 10.5294/edu.2018.21.1.5
6263 10.5294/edu.2018.21.1.6
6264 10.5294/edu.2017.20.3.1
6265 10.5294/edu.2017.20.3.2
6266 10.5294/edu.2017.20.3.3
6267 10.5294/edu.2017.20.3.4
6268 10.5294/edu.2017.20.3.5
6269 10.5294/edu.2017.20.3.6
6270 10.5294/edu.2017.20.3.7
6271 10.17230/ingciencia.12.24.1
6272 10.17230/ingciencia.13.26.1
6273 10.17230/ingciencia.13.26.2
6274 10.17230/ingciencia.13.26.3
6275 10.17230/ingciencia.13.26.4
6276 10.17230/ingciencia.13.26.5
6277 10.17230/ingciencia.13.26.6
6278 10.17230/ingciencia.13.26.7
6279 10.17230/ingciencia.13.26.8
6280 10.17230/ingciencia.13.26.9
6281 10.17230/ingciencia.13.26.10
6282 10.17230/ingciencia.14.27.1
6283 10.17230/ingciencia.14.27.2
6284 10.17230/ingciencia.14.27.3
6285 10.17230/ingciencia.14.27.4
6286 10.17230/ingciencia.14.27.5
6287 10.17230/ingciencia.14.27.6
6288 10.17230/ingciencia.14.27.7
6289 10.17230/ingciencia.14.27.8
6290 10.17230/ingciencia.14.27.9
6291 10.1590/1982-7849rac2018160078
6292 10.1590/1982-7849rac2018160123
6293 1

6499 10.15448/1981-2582.2018.1.26209
6500 10.15448/1981-2582.2018.1.27605
6501 10.15448/1981-2582.2018.1.25548
6502 10.15448/1981-2582.2018.1.24727
6503 10.15448/1981-2582.2018.1.24967
6504 10.15448/1981-2582.2018.1.24339
6505 recepción: 7 de septiembre de 2010 Fecha de aceptación: 21 de septiembre de 2010
6506 http://dx.doi.org/10.20983/noesis.2017.2.1
6507 http://dx.doi.org/10.20983/noesis.2017.2.2
6508 http://dx.doi.org/10.20983/noesis.2017.2.3
6509 http://dx.doi.org/10.20983/noesis.2017.2.4
6510 http://dx.doi.org/10.20983/noesis.2017.2.5
6511 http://dx.doi.org/10.20983/noesis.2017.2.6
6512 http://dx.doi.org/10.20983/noesis.2017.2.7
6513 http://dx.doi.org/10.20983/noesis.2017.2.8
6514 http://dx.doi.org/10.20983/noesis.2017.2.9
6515 http://dx.doi.org/10.20983/noesis.2018.1.1
6516 https://dx.doi.org/10.20983/noesis.2018.1.3
6517 https://dx.doi.org/10.20983/noesis.2018.1.3
6518 https://dx.doi.org/10.20983/noesis.2018.1.4
6519 http://dx.doi.org/10.20983/noesis.2018.1.5
6520 https://dx.d

6696 https://doi.org/10.18800/anthropologica.201702.006
6697 https://doi.org/10.18800/anthropologica.201702.007
6698 https://doi.org/10.18800/anthropologica.201702.008
6699 https://doi.org/10.18800/anthropologica.201702.009
6700 https://doi.org/10.18800/anthropologica.201702.009
6701 https://doi.org/10.18800/anthropologica.201801.001
6702 https://doi.org/10.18800/anthropologica.201801.002
6703 https://doi.org/10.18800/anthropologica.201801.004
6704 https://doi.org/10.18800/anthropologica.201801.007
6705 https://doi.org/10.18800/anthropologica.201801.005
6706 https://doi.org/10.18800/anthropologica.201801.006
6707 https://doi.org/10.18800/anthropologica.201801.007
6708 https://doi.org/10.18800/anthropologica.201801.008
6709 https://doi.org/10.18800/anthropologica.201801.009
6710 https://doi.org/10.18800/anthropologica.201801.010
6711 https://doi.org/10.18800/anthropologica.201801.011
6712 https://doi.org/10.18800/anthropologica.201801.012
6713 https://doi.org/10.18800/anthropologica.201

6874 https://doi.org/10.25267/RevEurekaensendivulgcienc.2019.v16.i1.1602
6875 https://doi.org/10.25267/RevEurekaensendivulgcienc.2019.v16.i1.1501
6876 https://doi.org/10.25267/RevEurekaensendivulgcienc.2019.v16.i1.1101
6877 https://doi.org/10.25267/RevEurekaensendivulgcienc.2019.v16.i1.1601
6878 https://doi.org/10.25267/RevEurekaensendivulgcienc.2019.v16.i1.1103
6879 https://doi.org/10.25267/RevEurekaensendivulgcienc.2019.v16.i1.1104
6880 https://doi.org/10.25267/RevEurekaensendivulgcienc.2019.v16.i1.1102
6881 https://doi.org/10.25267/RevEurekaensendivulgcienc.2019.v16.i1.1202
6882 https://doi.org/10.25267/RevEurekaensendivulgcienc.2019.v16.i1.1603
6883 https://doi.org/10.25267/RevEurekaensendivulgcienc.2019.v16.i1.1203
6884 https://doi.org/10.25267/RevEurekaensendivulgcienc.2019.v16.i1.1604
6885 https://doi.org/10.25267/RevEurekaensendivulgcienc.2019.v16.i1.1001
6886 https://doi.org/10.25267/RevEurekaensendivulgcienc.2019.v16.i1.1502
6887 https://doi.org/orcid.org/0000-0002-5239-2670


7127 10.17533/udea.iee.v36n3a09
7128 10.17533/udea.iee.v36n3a10
7129 https://doi.org/10.21500/22563202.3183
7130 http://dx.doi.org/10.5902/1984644424598
7131 http://dx.doi.org/10.5902/1984644417994
7132 http://dx.doi.org/10.5902/1984644415506
7133 http://dx.doi.org/10.5902/1984644423373
7134 http://dx.doi.org/10.5902/1984644418995
7135 http://dx.doi.org/10.5902/1984644423820  
7136 http://dx.doi.org/10.5902/1984644417438
7137 http://dx.doi.org/10.5902/1984644417918
7138 http://dx.doi.org/10.5902/1984644422612
7139 http://dx.doi.org/10.5902/1984644423022
7140 http://dx.doi.org/10.5902/1984644417591
7141 http://dx.doi.org/10.5902/1984644422100
7142 http://dx.doi.org/10.5902/1984644418924
7143 http://dx.doi.org/10.5902/1984644418036
7144 http://dx.doi.org/10.5902/1984644417694
7145 http://dx.doi.org/10.5902/1984644425247
7146 http://dx.doi.org/10.5902/1984644426372
7147 http://dx.doi.org/10.5902/1984644420116
7148 http://dx.doi.org/10.5902/1984644421950
7149 http://dx.doi.org/10.5902/1984

7328 https://doi.org/10.15446/achsc.v45n1.67563
7329 https://doi.org/10.15446/achsc.v45n1.67568
7330 https://doi.org/10.15446/achsc.v45n1.67575
7331 https://doi.org/10.15446/achsc.v45n1.67570
7332 https://doi.org/10.15446/achsc.v45n1.67571
7333 https://doi.org/10.15446/achsc.v45n1.67572
7334 https://doi.org/10.15446/achsc.v45n1.67573
7335 https://doi.org/10.15446/achsc.v45n2.71025
7336 https://doi.org/10.15446/achsc.v45n2.71026
7337 https://doi.org/10.15446/achsc.v45n2.71027
7338 https://doi.org/10.15446/achsc.v45n2.71028
7339 https://doi.org/10.15446/achsc.v45n2.71029
7340 https://doi.org/10.15446/achsc.v45n2.71030
7341 https://doi.org/10.15446/achsc.v45n2.71031
7342 https://doi.org/10.15446/achsc.v45n2.71032
7343 https://doi.org/10.15446/achsc.v45n2.71033
7344 https://doi.org/10.15446/achsc.v45n2.71034
7345 https://doi.org/10.15446/achsc.v45n2.71035
7346 https://doi.org/10.15446/achsc.v45n2.71036
7347 https://doi.org/10.15446/achsc.v45n2.71037
7348 https://doi.org/10.15446/achsc.v45n